In [ ]:
## Using Conv1D network on time series for classification 

In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
from scipy import signal
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle as pkl
import itertools 
import glob
from sklearn import svm 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, f1_score
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

%matplotlib inline 
# %matplotlib qt


In [3]:
#Import datasets
think_df = pd.read_pickle("F:\EEG-data\\think-count\splits\\0.1s/raw_think.pkl")
count_df = pd.read_pickle("F:\EEG-data\\think-count\splits\\0.1s/raw_count.pkl")

print(think_df.shape)
print(count_df.shape)
print(think_df.iloc[0,0].shape)
print(count_df.iloc[0,0].shape)

(320, 32)
(320, 32)
(12,)
(12,)


In [42]:
# Shape each dataframe into 320,12,32
def reshape_df(df):
    new_df = np.zeros((320,12,32))
    for i in range(32):
        channel = df.iloc[:,i].values 
        channel_df = np.zeros((320,12))
        for j in range(len(channel)):
            channel_df[j,:] = channel[j]
        new_df[:,:,i] = keras.utils.normalize(channel_df)
    return new_df

In [43]:
r_think_df = reshape_df(think_df)
r_count_df = reshape_df(count_df)

In [44]:
X = np.vstack((r_think_df,r_count_df))
y = np.hstack((np.zeros(r_think_df.shape[0]),np.ones(r_count_df.shape[0])))
print(X.shape)
print(y.shape)

(640, 12, 32)
(640,)


In [45]:
#Split into train and test sets
def split_train_test(X,y):
    sss = StratifiedShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for train_index, test_index in sss.split(X,y):
                x_train, x_test = X[train_index],X[test_index]
                y_train, y_test = y[train_index],y[test_index]
    return x_train,x_test,y_train,y_test

In [46]:
x_train,x_test,y_train,y_test = split_train_test(X,y)

In [48]:
print(x_train.shape)
print(x_test.shape)

(512, 12, 32)
(128, 12, 32)


In [50]:
#DNN model

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12*32, input_shape = (12,32), activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"), 
    tf.keras.layers.Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 12, 384)           12672     
_________________________________________________________________
dense_4 (Dense)              (None, 12, 10)            3850      
_________________________________________________________________
dense_5 (Dense)              (None, 12, 1)             11        
Total params: 16,533
Trainable params: 16,533
Non-trainable params: 0
_________________________________________________________________


In [75]:
dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
dataset = dataset.batch(1)

model.fit(dataset,epochs=2)


Epoch 1/2
512/512 [==============================] - 2s 3ms/step - loss: 0.0538 - accuracy: 0.9780
Epoch 2/2
512/512 [==============================] - 2s 3ms/step - loss: 0.0350 - accuracy: 0.9884


In [76]:
loss,accuracy = model.evaluate(x_test,y_test) 

print(accuracy)

4/4 [==============================] - 0s 2ms/step - loss: 0.3743 - accuracy: 0.9108
0.9108072519302368
